<a href="https://www.kaggle.com/code/vikramsandu/dl-nlp-commonlit-summaries?scriptVersionId=138250613" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Goal

* The immediate goal is to setup this notebook for training, Inference and submission for LB evaluation. <font color="green">&#10004;</font>

* Efficiently Back Translation of the Data for Augmentations. <font color="green">&#10004;</font>


Let's Create a Controller variable for Training/Submission Mode.

* Set **True** for Submission Mode.
* Set **False** for Training Mode.

In [1]:
# True if you want to submit the Notebook
isSubmit = True

if isSubmit:
    print(f'This notebook is on submission Mode...')
    print(f'Make sure to turn off the Internet...')
else:
    print(f'This notebook is on Training Mode...')

This notebook is on submission Mode...
Make sure to turn off the Internet...


### Read and Understand Data.

Here we have two type of files: 
1. **<span style="color:red">Prompts:</span>** It contains the Question, a title about the text, and the text that needs to be summarized.
2. **<span style="color:red">Summaries:</span>** This includes the text summarized by the students, along with the corresponding prompt id and target scores for both content and wording.

In [2]:
# Read Dataset
import pandas as pd

if not isSubmit:
    # Train
    prompt_df = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv')
    summary_df = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv')

    print(f'\nLength of Train Prompt df: {len(prompt_df)}')
    print(f'Length of Train Summary df: {len(summary_df)}\n')

    # Display
    display(summary_df.sample(5))

In [3]:
if not isSubmit:
    # Distribution of the Prompt Questions.
    print(summary_df['prompt_id'].value_counts())

In [4]:
if not isSubmit:
    # Number of Unique Students
    print(len(summary_df['student_id'].unique()))

**So the Training data contains only 4 Prompt Questions that is being asked to 7165 students.**

Now Let's understand the Labels.

In [5]:
if not isSubmit: 
    # Describe
    describe_content_df = summary_df['content'].describe()
    describe_wording_df = summary_df['wording'].describe()

    print(pd.concat([describe_content_df, describe_wording_df], axis=1))

* The value of **Content** ranging from -1.72 to 3.90.
* Value of the **Wording** is ranging from -1.96 to 4.31.

It surely has two classes (Content and Wording) and each class has a continuous output. This problem can be put into the category of **Two-Class Regression.**

Let's have an overall idea about the number of words (or tokens) in summary text.

In [6]:
if not isSubmit:
    text_length = summary_df['text'].apply(lambda x: len(x.split(' ')))
    print(text_length.describe())
    print('\n')

    # Lets Visualize the same.
    from matplotlib import pyplot as plt

    # Create a histogram using Matplotlib
    plt.figure(figsize = (3, 3))
    plt.hist(text_length, bins=10, edgecolor='k')
    plt.xlabel('Text Length')
    plt.ylabel('Frequency')
    plt.title(f'Distribution of the Text Length')

The average text summary length is approximately 76 tokens, with a standard deviation of 54 tokens. Most of the text summaries have a length of less than 200 tokens. That is still in the limit of **BERT** and related models, which accept a **maximum of 512 tokens**. In Future, we might use some of the contextual information from the Prompt text to utilize this token gap properly.

Let's have an overall idea about the number of words (or tokens) in Prompt Text.

In [7]:
if not isSubmit:
    text_length = prompt_df['prompt_text'].apply(lambda x: len(x.split(' ')))
    # Lets Visualize the same.
    print(text_length)

Prompt Text is ranging from 550 to 966 words.

**We must provide this contextual information (prompt text) along with the text summary to yield better results.**

But this will result in a longer document (way longer than transformer can process --512 tokens--), to tackle this we will use longformer.

* Transformer-based models are unable to process long sequences due to their self-attention operation, which scales quadratically with the sequence length. To address this limitation, we introduce the Longformer with an attention mechanism that scales linearly with sequence length, making it easy to process documents of thousands of tokens or longer. 

we'll get back to this later if needed.

## Data Augmentation

**Back Translation DA:**

English ---> French/Spanish/Hindi/etc ---> English

The output English sentence will differ a little bit from the input sentence.

In [8]:
isRunOnlyOnce = True

if not isRunOnlyOnce:
    import torch
    from torch.utils.data import Dataset
    from transformers import MarianTokenizer

    class TranslationDataset(Dataset):
        def __init__(self, 
                     summary_df,
                     model_name
                    ):
            self.summary_df = summary_df
            self.tokz = MarianTokenizer.from_pretrained(model_name)

        def __len__(self):
            return len(self.summary_df)

        def __getitem__(self, idx):

            # Get the Summary and It's Corresponding Question
            txt_summary = self.summary_df['text'].iloc[idx] 
            encodings = self.tokz.encode_plus(txt_summary, 
                                              add_special_tokens=True, 
                                              max_length = 512, 
                                              padding = 'max_length', 
                                              truncation = True, 
                                              return_tensors = 'pt'
                                             )

            input_ids = encodings['input_ids'].squeeze()
            attention_mask = encodings['attention_mask'].squeeze()

            return {
                'input_ids': input_ids,
                'attention_mask': attention_mask
            }

**English to French Conversion**

In [9]:
if not isRunOnlyOnce:
    # Initialize Dataset Classes
    model_name = "Helsinki-NLP/opus-mt-en-fr"

    commonlit_summary_translation_ds = TranslationDataset(summary_df, model_name)
    print(f'Data - {len(commonlit_summary_translation_ds)}')

    # Dataloader
    from torch.utils.data import DataLoader

    # Create a data loader for the dataset
    batch_size = 16
    dataloader = DataLoader(commonlit_summary_translation_ds, 
                            batch_size=batch_size, 
                            shuffle=False
                           )

    # Inference
    from tqdm import tqdm
    from torch.nn.parallel import DataParallel
    from transformers import MarianMTModel
    from torch.cuda.amp import autocast

    model_name = "Helsinki-NLP/opus-mt-en-fr"
    tokz = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)

    # Check if GPU is available.
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'DEVICE: {device}\n')

    model = model.to(device)
    model = DataParallel(model)

    french_summary_txts = []
    for batch in tqdm(dataloader):
        with autocast():
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            actual_model = model.module
            outputs = actual_model.generate(input_ids = input_ids, 
                                            attention_mask = attention_mask)
            translated_text = tokz.batch_decode(outputs, skip_special_tokens=True)
            french_summary_txts += translated_text

    french_summary_df = pd.DataFrame(french_summary_txts, columns=['text'])
    french_summary_df.tail()

**French to English Translation**

In [10]:
if not isRunOnlyOnce:
    # Initialize Dataset Classes
    model_name = "Helsinki-NLP/opus-mt-fr-en"

    commonlit_summary_translation_ds = TranslationDataset(french_summary_df, model_name)
    print(f'Data - {len(commonlit_summary_translation_ds)}')

    # Create a data loader for the dataset
    batch_size = 16
    dataloader = DataLoader(commonlit_summary_translation_ds, 
                            batch_size=batch_size, 
                            shuffle=False
                           )

    model_name = "Helsinki-NLP/opus-mt-fr-en"
    tokz = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)

    # Check if GPU is available.
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'DEVICE: {device}\n')

    model = model.to(device)
    model = DataParallel(model)

    summary_bt_txts = []
    for batch in tqdm(dataloader):
        with autocast():
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            actual_model = model.module
            outputs = actual_model.generate(input_ids = input_ids, 
                                            attention_mask = attention_mask)
            translated_text = tokz.batch_decode(outputs, skip_special_tokens=True)
            summary_bt_txts += translated_text

    summary_BT_df = pd.DataFrame(summary_bt_txts, columns=['text'])
    summary_BT_df.tail()

In [11]:
if not isRunOnlyOnce:
    # Save
    summary_BT_df.to_csv('BT_french_summary_df.csv', index=False)

### Dataset and Dataloaders

Now Let's define the Dataset Class.

The Dataset class in Natural Language Processing (NLP) serves as a fundamental data structure that helps manage and handle textual data for training and evaluation purposes.

In [12]:
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer
import numpy as np

class CommonLitSummaryDataset(Dataset):
    def __init__(self, 
                 summary_df,
                 prompt_df, 
                 model_name, 
                 max_length = 256,
                 isTest = False,
                 aug = False
                ):
        
        if not isTest:
            self.orig_summary_df = summary_df[summary_df['BT_lang']=='English'].reset_index(drop=True)
        else:
            self.orig_summary_df = summary_df
        
        if aug:
            self.BT_summary_df = summary_df[summary_df['BT_lang']!='English'].reset_index(drop=True)
            
        self.prompt_df = prompt_df
        self.max_length = max_length
        self.tokz = AutoTokenizer.from_pretrained(model_name)
        self.isTest = isTest
        self.aug = aug
       
    def __len__(self):
        return len(self.orig_summary_df)
    
    def __getitem__(self, idx):
        
        # Get the Summary and It's Corresponding Question
        txt_summary = self.orig_summary_df['text'].iloc[idx]
        
        if self.aug:
            if np.random.uniform() <= 0.25:
                txt_summary = self.BT_summary_df['text'].iloc[idx]
        
        prompt_id = self.orig_summary_df['prompt_id'].iloc[idx]
        prompt_question = self.prompt_df[self.prompt_df['prompt_id'] == prompt_id]['prompt_question'].iloc[0]
        
        # Concat the Question and Summary.
        input_text = 'QUESTION: ' + prompt_question + 'SUMMARY: ' + txt_summary 
        
        # Convert the text data into Corresponding Numerical Embeddings.
        encodings = self.tokz.encode_plus(input_text, 
                                          add_special_tokens=True, 
                                          max_length = self.max_length, 
                                          padding = 'max_length', 
                                          truncation = True, 
                                          return_tensors = 'pt'
                                         )
        input_ids = encodings['input_ids'].squeeze()
        attention_mask = encodings['attention_mask'].squeeze()
        
        # For Test set, No labels will be available
        if self.isTest:
            return {
            'input_ids': input_ids,
            'attention_mask': attention_mask
        }
            
        #label = torch.tensor(self.summary_df.iloc[idx][-2:].tolist())
        label = torch.tensor([self.orig_summary_df.iloc[idx]['content'], self.orig_summary_df.iloc[idx]['wording']])
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': label
        }

Before Initialising the Class, We have to split the dataset into two categories. 1) Train 2) Valid

In [13]:
if not isSubmit:
    
    isStratified = False
    
    # Split the dataframe into train and validation sets
    from sklearn.model_selection import train_test_split
    
    if not isStratified:
        # Normal Split
        train_df, valid_df = train_test_split(summary_df, test_size=0.2, random_state=42)
       # train_df = train_df.reset_index(drop = True)
       # valid_df = valid_df.reset_index(drop = True)
    else:
        # Stratified Split
        test_prompt_id = ['3b9047']
        
        # Filter the DataFrame to create train and test sets
        train_df = summary_df[~summary_df['prompt_id'].isin(test_prompt_id)].reset_index(drop=True)
        valid_df = summary_df[summary_df['prompt_id'].isin(test_prompt_id)].reset_index(drop=True)

    
    print('Train Prompt IDs: ', train_df['prompt_id'].unique())
    print('Valid Prompt IDs: ', valid_df['prompt_id'].unique())

In [14]:
if not isSubmit:
    from copy import deepcopy
    fr_summary_df = pd.read_csv('/kaggle/input/commonlit-summaries-additional-datasets/BT_french_summary_df.csv')
    fr_summary_df = fr_summary_df.iloc[train_df.index]
    FR_train_df = deepcopy(train_df)
    FR_train_df['text'] =  fr_summary_df['text'].tolist()
    FR_train_df['BT_lang'] = 'French'
    FR_train_df = FR_train_df.reset_index(drop=True)

    # Original Train DF
    train_df['BT_lang'] = 'English'
    train_df = train_df.reset_index(drop=True)

    # Consolidated
    consolidated_train_df = pd.concat([train_df, FR_train_df], axis=0).reset_index(drop=True)
    consolidated_train_df.tail()

In [15]:
MAX_LENGTH = 256
MODEL_NAME = '/kaggle/input/commonlit-summaries-all-tokenizers/bert_base_cased_tokenizer'

if not isSubmit:
    
    # Initialize Dataset Classes
    commonlit_summary_train_ds = CommonLitSummaryDataset(consolidated_train_df,
                                                         prompt_df, 
                                                         model_name = MODEL_NAME, 
                                                         max_length = MAX_LENGTH,
                                                        # aug = True
                                                        )
    
    valid_df = valid_df.reset_index(drop=True)
    valid_df['BT_lang'] = 'English'
    commonlit_summary_valid_ds = CommonLitSummaryDataset(valid_df, 
                                                         prompt_df,  
                                                         model_name = MODEL_NAME, 
                                                         max_length = MAX_LENGTH
                                                        )
    print(f'Train - {len(commonlit_summary_train_ds)}, Test - {len(commonlit_summary_valid_ds)}')

Let's Visualize one Sample to see the working of our Dataset.

In [16]:
if not isSubmit:

    # Tokenizer
    tokz =  AutoTokenizer.from_pretrained(MODEL_NAME)

    print(f'------ Input ----------\n')
    sample = commonlit_summary_train_ds[0]
    print(tokz.decode(sample['input_ids']))

    print(f'\n------ Labels ----------\n')
    labels = sample['labels']
    print(labels)

**Dataloader:**

Dataloader allows us to group multiple samples into batches, enabling parallel processing and more efficient GPU utilization during training.

In [17]:
# Dataloader
from torch.utils.data import DataLoader

if not isSubmit:
    # Create a data loader for the dataset
    batch_size = 16
    train_dataloader = DataLoader(commonlit_summary_train_ds, batch_size=batch_size, shuffle=True)
    eval_dataloader = DataLoader(commonlit_summary_valid_ds, batch_size=batch_size, shuffle=False)

### Model Training

In [18]:
from transformers import AutoModelForSequenceClassification, AdamW

model_nm = 'bert-base-cased'
num_labels = 2

if not isSubmit:
    model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=num_labels)
    
    # Increase the dropout
    model.dropout = torch.nn.Dropout(p=0.5, inplace=False)

    total_params = sum(p.numel() for p in model.parameters())
    print("\nTotal number of parameters: ", total_params)

    total_size = sum(p.numel() * p.element_size() for p in model.parameters())
    print("Total size (bytes) of the model: ", total_size)
    print("Total size (MB) of the model: ", total_size / (1024 * 1024))

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

we need to save the best model during Training based on the **MCRMSE** (mean columnwise root mean squared error) which is the Cost Function/ Metric for this Competition.

Let's implement this.

In [19]:
# Utility: mean columnwise root mean squared error
def mcrmse_loss(predictions, targets):
    rmse_columnwise = torch.sqrt(torch.mean((predictions - targets)**2, dim=0))
    return torch.mean(rmse_columnwise)

**Training Loop**

In order to have **faster Training**, we will do the followings:

1. **Enable mixed precision training** (using half-precision floating-point format or fp16). It utilizes Tensor Cores on supported NVIDIA GPUs to speed up the training process with reduced memory usage.

2.  To utilize Kaggle T4/X2 GPU, we will use **DataParallel**. DataParallel allows you to train your model on multiple GPUs simultaneously, distributing the workload across all available GPUs.

In [20]:
if not isSubmit:
    
    # Import Dataparallel and mixed precision modules
    from torch.cuda.amp import autocast, GradScaler
    from torch.nn.parallel import DataParallel

    # Check if GPU is available.
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'DEVICE: {device}\n')

    # Used for Mixed Precision Training
    scaler = GradScaler()

    # Move your model to the GPU and wrap it with DataParallel (To utilize T4X2)
    model = model.to(device)
    model = DataParallel(model) 

Let's Train the Model.

In [21]:
# Visualize progress bar
from tqdm import tqdm

if not isSubmit:
    # Prepare optimizer
    optimizer = AdamW(model.parameters(), lr=1e-5)
    NUM_EPOCHS = 15

    best_eval_loss = float('inf')  # Initialize the best evaluation loss to infinity

    for epoch in range(NUM_EPOCHS):
        total_loss = 0
        model.train() # Set the model to Training mode

        for batch in tqdm(train_dataloader):

            with autocast():
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                # Forward loop
                optimizer.zero_grad() # Ensures Gradient doesn't accumulate.
                predictions = model(input_ids=input_ids,
                                    attention_mask=attention_mask, 
                                    labels=labels
                                   ).logits

                # Compute MCRMSE loss
                loss = mcrmse_loss(predictions, labels)

            # BackProp
            scaler.scale(loss).backward()  # Scale the loss value
            scaler.step(optimizer)
            scaler.update()

            # Accumulate the Loss
            total_loss += loss.item()

        # Calculate epoch-level metrics
        epoch_loss = total_loss / len(train_dataloader)

        # Print epoch-level metrics
        print(f"Epoch {epoch+1}/{NUM_EPOCHS}")
        print(f"Train Loss: {epoch_loss:.4f}")

         # Evaluation
        model.eval()  # Set model to evaluation mode
        eval_loss = 0

        with torch.no_grad():
            for batch in eval_dataloader:
                with autocast():
                    input_ids = batch['input_ids'].to(device)
                    attention_mask = batch['attention_mask'].to(device)
                    labels = batch['labels'].to(device)

                    predictions = model(input_ids=input_ids,
                                        attention_mask=attention_mask, 
                                        labels=labels
                                       ).logits
                    loss = mcrmse_loss(predictions, labels)

                    eval_loss += loss.item()

        # Calculate evaluation metrics
        eval_epoch_loss = eval_loss / len(eval_dataloader)

        # Print evaluation metrics
        print(f"Eval Loss: {eval_epoch_loss:.4f}")

        # Save the Best Model
        if eval_epoch_loss < best_eval_loss:
            print(f'--------------------------------------')
            print(f'Found the best model at Epoch {epoch+1}')
            print(f'Validation Loss reduced from {best_eval_loss:.4f} to {eval_epoch_loss:.4f}')
            best_eval_loss = eval_epoch_loss
            print(f'Saving the best model.')
            print(f'--------------------------------------\n')
            
            # Access the actual model from the DataParallel object
            actual_model = model.module
            # Save
            actual_model.save_pretrained("bert_base_cased_256")

### Model Inference

Load the Best model.

In [22]:
model_path = '/kaggle/input/commonlit-summaries-all-models/bert_base_cased_256/bert_base_cased_256'
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Check if GPU is available.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'DEVICE: {device}\n')

# Model to the Device
model = model.to(device)

DEVICE: cuda



Prepare Test Data Loader.

In [23]:
# Read Test DF's.
prompt_test_df = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv')
summary_test_df = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv')

# Initialize Dataset Classes
commonlit_summary_test_ds = CommonLitSummaryDataset(summary_test_df,
                                                    prompt_test_df, 
                                                    model_name = MODEL_NAME, 
                                                    max_length = MAX_LENGTH,
                                                    isTest = True
                                                    )
# Test Dataloader
batch_size = 16
test_loader = DataLoader(commonlit_summary_test_ds, batch_size=batch_size, shuffle=False)

test_loader

**Prediction**

In [24]:
# Eval model
model.eval()

predictions = []
with torch.no_grad():  # Disable gradient computation during inference
    for batch in tqdm(test_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        
        # Predictions
        predictions_batch = model(input_ids=input_ids,
                                  attention_mask=attention_mask
                                 ).logits
        
        # Collect predictions from this batch
        predictions.extend(predictions_batch.cpu().tolist())
        
        
# Convert to numpy
import numpy as np
predictions = np.array(predictions)

100%|██████████| 1/1 [00:02<00:00,  2.63s/it]


**Submission**

In [25]:
# Create a DataFrame
data = {
    'student_id': summary_test_df['student_id'].tolist(),
    'content': predictions[:,0],
    'wording': predictions[:,1]
}
submission_df = pd.DataFrame(data)

# Display
display(submission_df.head())

# Save it for Submission
submission_df.to_csv('submission.csv', index=False)

,student_id,content,wording
0,000000ffffff,-1.720263,-1.530592
1,111111eeeeee,-1.735618,-1.543903
2,222222cccccc,-1.728401,-1.549866
3,333333dddddd,-1.734978,-1.543963


**TODO:**

* FP16 Training <font color="green">&#10004;</font>
* Utilize Both of the GPU's <font color="green">&#10004;</font>
* Stratified Split <font color="green">&#10004;</font>
* Data Augmentations
* Model Architecture Tweaking

**Model Performance Tracker**

<style>
table {
    width: 100%;
    border-collapse: collapse;
}

th, td {
    padding: 8px;
    text-align: left;
}

th {
    background-color: #FF0000; /* Red color */
    color: white;
}
</style>

<table>
    <tr>
        <th><span style="color:red">S.No.</span></th>
        <th><span style="color:red">Seed/ID</span></th>
        <th><span style="color:red">Split</span></th>
        <th><span style="color:red">Model_name</span></th>
        <th><span style="color:red">Input Tokens</span></th>
        <th><span style="color:red">DA</span></th>
        <th><span style="color:red">Others</span></th>
        <th><span style="color:red">CV</span></th>
        <th><span style="color:red">LB</span></th>
    </tr>
    <tr>
        <td>1</td>
        <td>42</td>
        <td>Random</td>
        <td>bert_base_cased_256</td>
        <td>256</td>
        <td>None</td>
        <td>NA</td>
        <td>0.475</td>
        <td>NA</td>
    </tr>
    <tr>
        <td>2</td>
        <td>3b9047</td>
        <td>Stratify</td>
        <td>vanilla_bert_base_cased_stratified_3b9047</td>
        <td>256</td>
        <td>None</td>
        <td>NA</td>
        <td>0.670</td>
        <td>0.549</td>
    </tr>
    <tr>
        <td>3</td>
        <td>42</td>
        <td>Random</td>
        <td>bert_base_cased_512</td>
        <td>512</td>
        <td>None</td>
        <td>NA</td>
        <td>0.465</td>
        <td>0.556</td>
    </tr>
    <tr>
        <td>4</td>
        <td>42</td>
        <td>Random</td>
        <td>bert_base_cased_256_BT_FR</td>
        <td>256</td>
        <td>BT(FR)</td>
        <td>NA</td>
        <td>0.478</td>
        <td>NA</td>
    </tr>
    
</table>


**Short Forms:**
* **BT (FR)** Back Translation (French)